# NB for Moving from Development Code to Production

## Class Breakdown

### Dask handlers

- Start/keep track of dask scheduler 
- Start/keep track of workers
- Use configuration 

### Dask Configuration

- Settings for various instrumentation: PM, CPU, GPU, etc.

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import ipywidgets as widgets
import logging
from IPython.display import display, HTML

class HTMLFormatter(logging.Formatter):
    """
    Prettily formats logs. Could use rich, too.
    From: https://stackoverflow.com/questions/68807282/rich-logging-output-in-jupyter-ipython-notebook
    """
    level_colors = {
        logging.DEBUG: 'lightblue',
        logging.INFO: 'dodgerblue',
        logging.WARNING: 'goldenrod',
        logging.ERROR: 'crimson',
        logging.CRITICAL: 'firebrick'
    }
    
    def __init__(self):
        super().__init__(
            '<span style="font-weight: bold; color: green">{asctime}</span> '
            '[<span style="font-weight: bold; color: {levelcolor}">{levelname}</span>] '
            '{message}',
            style='{'
        )
    
    def format(self, record):
        record.levelcolor = self.level_colors.get(record.levelno, 'black')
        return HTML(super().format(record))

class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        layout = {
            'width': '100%', 
        }
        self.setFormatter(HTMLFormatter())
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        self.out.append_display_data(formatted_record)
        
    def show_logs(self):
        """ Show the logs """
        display(self.out)
    
    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()

In [3]:
def create_new_cell(contents):
    """
    Create new notebook cell programatically.
    https://stackoverflow.com/questions/54987129/how-to-programmatically-create-several-new-cells-in-a-jupyter-notebook
    """
    from IPython.core.getipython import get_ipython
    shell = get_ipython()

    payload = dict(
        source='set_next_input',
        text=contents,
        replace=False,
    )
    shell.payload_manager.write_payload(payload, single=False)

In [4]:
import dask
from dask.distributed import Client
import os
import logging

class SystemConfig:
    
    def __init__(self):
        self.configure()
        
    def _determine_system(self):
        try:
            self.system = os.environ["NERSC_HOST"]
        except KeyError:
            self.system = "local"
    
    def _determine_ipython(self):
        """
        Checks if this is running within a jupyter notebook.
        """
        shell = get_ipython().__class__.__name__
        try:
            if shell == "ZMQInteractiveShell":
                self.shell = "jupyter"
                self._in_jupyter = True
            elif shell == "TerminalInteractiveShell":
                self.shell = "iPython"
                self._in_ipython = True
            else:
                self.shell = "unknown"
        except NameError:
            self.shell = "standard_interpreter"
            self._in_standard = True
            
    def _create_logger(self, ):
        self.log = logging.getLogger("system")
        if self._in_jupyter or self._in_ipython:
            self.log_handler = OutputWidgetHandler()
            self.log.addHandler(self.log_handler)
            self.log.setLevel(logging.INFO)
        else:
            self.log.setLevel(logging.INFO)
            
    def configure(self):
        self._determine_system()
        self._determine_ipython()
        self._create_logger()
        self.log.info(f"You are using {self.system}")
        self.log.info(f"You are running python inside of {self.shell}")    

x = SystemConfig()
# x.log_handler.show_logs()

In [5]:

configuration_defaults_yml = {

}

class DaskConfig:
    
    def __init__(self, sys_config: SystemConfig):
        self.sys_config = sys_config
        self.configure()
        
    def _create_logger(self):
        self.log = logging.getLogger("system.dask")
        if self.sys_config._in_jupyter or self.sys_config._in_ipython:
            self.log_handler = OutputWidgetHandler()
            self.log.addHandler(self.log_handler)
            self.log.setLevel(logging.INFO)
        else:
            self.log.setLevel(logging.INFO)
            
    def configure(self):
        self._create_logger()
        self.log.info(f"Starting dask configuration")

y = DaskConfig(x)
y.log_handler.show_logs()

import shlex
import logging
import subprocess

def log_subprocess_output(pipe, logger=y.log):
    for line in iter(pipe.readline, b''): # b'\n'-separated lines
        logger.info(line)
        
from subprocess import Popen, PIPE, STDOUT

process = Popen("/global/homes/s/swelborn/stempy-dask/start_dask.sh", stdout=PIPE, stderr=STDOUT)
with process.stdout:
    log_subprocess_output(process.stdout)
exitcode = process.wait() # 0 means success


Output(layout=Layout(width='100%'), outputs=({'output_type': 'display_data', 'data': {'text/plain': '<IPython.…

In [19]:
a = DaskConfig()


INFO:root:Set system to cori


### Data operations

- Loading/counting data
- Tracking scan location
- Saving data
- Creating stempy reader